In [ ]:
import numpy as np#model1
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
file_path = '/content/morning cluster.sx.xlsx'  # Update this with your actual file path
data = pd.read_excel(file_path)
data['Load_Lag_1'] = data['POWER (KW)'].shift(1)     # Previous hour
data['Load_Lag_24'] = data['POWER (KW)'].shift(24)   # Previous day
data['Load_Lag_168'] = data['POWER (KW)'].shift(168) # Previous week

# Create lagged features for seasonal parameters (example: Temperature and Humidity)
data['Temp_Lag_1'] = data['Temp (F)'].shift(1)
data['Temp_Lag_24'] = data['Temp (F)'].shift(24)
data['Humidity_Lag_1'] = data['Humidity (%)'].shift(1)
data['Humidity_Lag_24'] = data['Humidity (%)'].shift(24)
data.dropna(inplace=True)
X = data[['Load_Lag_1','Load_Lag_24','Load_Lag_168','Temp_Lag_1','"WEEKEND/WEEKDAY"','SEASON','Temp_Lag_24','Humidity_Lag_1','Humidity_Lag_24']]
y = data['POWER (KW)']
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled_morning = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100]},
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto'], 'coef0': [0, 1]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 0.01]},
    {'kernel': ['sigmoid'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'coef0': [0, 1]}
]

# Initialize the SVR model
svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train_scaled, y_train_scaled)

# Get the best estimator
best_svr = grid_search.best_estimator_
y_pred_train_scaled_morning = best_svr.predict(X_train_scaled)
y_pred_test_scaled_morning = best_svr.predict(X_test_scaled)

In [ ]:
import pandas as pd  #model2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import math
data = pd.read_excel('/content/afternoon cluster.xlsx')
# Select features for the model
features = ['POWER (KW)', 'Temp (F)', 'Humidity (%)', '"WEEKEND/WEEKDAY"','SEASON']
# Scale the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[features])

# Convert the scaled data back to a DataFrame
scaled_data = pd.DataFrame(scaled_data, columns=features, index=data.index)
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data.iloc[i + seq_length, 0]  # Load is the target variable
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 24  # Using 24 hours of data to predict the next hour's load
X, y = create_sequences(scaled_data, seq_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(seq_length, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, batch_size=15, epochs=30, validation_split=0.2)
predictions_afternoon = model.predict(X_test)

Epoch 1/30
62/62 [==============================] - 13s 82ms/step - loss: 0.0214 - val_loss: 0.0120
Epoch 2/30
62/62 [==============================] - 3s 55ms/step - loss: 0.0112 - val_loss: 0.0102
Epoch 3/30
62/62 [==============================] - 5s 81ms/step - loss: 0.0097 - val_loss: 0.0087
Epoch 4/30
62/62 [==============================] - 3s 56ms/step - loss: 0.0096 - val_loss: 0.0092
Epoch 5/30
62/62 [==============================] - 3s 54ms/step - loss: 0.0099 - val_loss: 0.0080
Epoch 6/30
62/62 [==============================] - 4s 63ms/step - loss: 0.0082 - val_loss: 0.0085
Epoch 7/30
62/62 [==============================] - 4s 70ms/step - loss: 0.0075 - val_loss: 0.0082
Epoch 8/30
62/62 [==============================] - 3s 53ms/step - loss: 0.0077 - val_loss: 0.0077
Epoch 9/30
62/62 [==============================] - 3s 53ms/step - loss: 0.0078 - val_loss: 0.0079
Epoch 10/30
62/62 [==============================] - 5s 83ms/step - loss: 0.0079 - val_loss: 0.0079
Epoch 11

In [ ]:
import pandas as pd  #model3
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
file_path = '/content/evening clusterxlsx.xlsx'  # replace with your file path
data = pd.read_excel(file_path)
data['Load_Lag_1'] = data['POWER (KW)'].shift(1)     # Previous hour
data['Load_Lag_24'] = data['POWER (KW)'].shift(24)   # Previous day
data['Load_Lag_168'] = data['POWER (KW)'].shift(168) # Previous week

# Create lagged features for seasonal parameters (example: Temperature and Humidity)
data['Temp_Lag_1'] = data['Temp (F)'].shift(1)
data['Temp_Lag_24'] = data['Temp (F)'].shift(24)
data['Humidity_Lag_1'] = data['Humidity (%)'].shift(1)
data['Humidity_Lag_24'] = data['Humidity (%)'].shift(24)
data.dropna(inplace=True)
features = ['Load_Lag_1', 'Load_Lag_24', 'Load_Lag_168', 'Temp_Lag_1', 'Temp_Lag_24', 'Humidity_Lag_1', 'Humidity_Lag_24','"WEEKEND/WEEKDAY"', 'SEASON']
# Scale the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[features])

# Convert the scaled data back to a DataFrame
scaled_data = pd.DataFrame(scaled_data, columns=features, index=data.index)
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data.iloc[i + seq_length, 0]  # Load is the target variable
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 24  # Using 24 hours of data to predict the next hour's load
X, y = create_sequences(scaled_data, seq_length)
X_train, X_test, y_train, y_test_eveing = train_test_split(X, y, test_size=0.2, random_state=42)
model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(seq_length, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, batch_size=15, epochs=30, validation_split=0.2)
predictions_evening = model.predict(X_test)

Epoch 1/30
55/55 [==============================] - 12s 83ms/step - loss: 0.0490 - val_loss: 0.0178
Epoch 2/30
55/55 [==============================] - 4s 74ms/step - loss: 0.0222 - val_loss: 0.0131
Epoch 3/30
55/55 [==============================] - 3s 59ms/step - loss: 0.0182 - val_loss: 0.0143
Epoch 4/30
55/55 [==============================] - 3s 53ms/step - loss: 0.0168 - val_loss: 0.0120
Epoch 5/30
55/55 [==============================] - 3s 54ms/step - loss: 0.0153 - val_loss: 0.0106
Epoch 6/30
55/55 [==============================] - 4s 76ms/step - loss: 0.0159 - val_loss: 0.0107
Epoch 7/30
55/55 [==============================] - 3s 62ms/step - loss: 0.0160 - val_loss: 0.0106
Epoch 8/30
55/55 [==============================] - 3s 56ms/step - loss: 0.0153 - val_loss: 0.0106
Epoch 9/30
55/55 [==============================] - 3s 56ms/step - loss: 0.0148 - val_loss: 0.0115
Epoch 10/30
55/55 [==============================] - 4s 78ms/step - loss: 0.0137 - val_loss: 0.0111
Epoch 11

In [ ]:
import numpy as np#model1
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
file_path = '/content/night cluster.xlsx'  # Update this with your actual file path
data = pd.read_excel(file_path)
data['Load_Lag_1'] = data['POWER (KW)'].shift(1)     # Previous hour
data['Load_Lag_24'] = data['POWER (KW)'].shift(24)   # Previous day
data['Load_Lag_168'] = data['POWER (KW)'].shift(168) # Previous week

# Create lagged features for seasonal parameters (example: Temperature and Humidity)
data['Temp_Lag_1'] = data['Temp (F)'].shift(1)
data['Temp_Lag_24'] = data['Temp (F)'].shift(24)
data['Humidity_Lag_1'] = data['Humidity (%)'].shift(1)
data['Humidity_Lag_24'] = data['Humidity (%)'].shift(24)
data.dropna(inplace=True)
X = data[['Load_Lag_1','Load_Lag_24','Load_Lag_168','Temp_Lag_1','"WEEKEND/WEEKDAY"','SEASON','Temp_Lag_24','Humidity_Lag_1','Humidity_Lag_24']]
y = data['POWER (KW)']
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled_morning = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100]},
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto'], 'coef0': [0, 1]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 0.01]},
    {'kernel': ['sigmoid'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'coef0': [0, 1]}
]

# Initialize the SVR model
svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train_scaled, y_train_scaled)

# Get the best estimator
best_svr = grid_search.best_estimator_
y_pred_train_scaled_morning = best_svr.predict(X_train_scaled)
y_pred_test_scaled_morning = best_svr.predict(X_test_scaled)
y_pred_test_scaled_night = best_svr.predict(X_test_scaled)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
file_path = '/content/godishala clusters removed.xlsx' # Update this with your actual file path
data = pd.read_excel(file_path)
X = data[['Temp (F)', 'Humidity (%)', '"WEEKEND/WEEKDAY"','SEASON']]
y = data['POWER (KW)']
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train, y_test_nocluster = train_test_split(X, y, test_size=0.2, random_state=0)
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled_nonclusterd = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100]},
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto'], 'coef0': [0, 1]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 0.01]},
    {'kernel': ['sigmoid'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'coef0': [0, 1]}
]

# Initialize the SVR model
svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train_scaled, y_train_scaled)

# Get the best estimator
best_svr = grid_search.best_estimator_
y_pred_train_scaled = best_svr.predict(X_train_scaled)
y_pred_test_scaled_nonclusterd_noclusterd= best_svr.predict(X_test_scaled)
